In [ ]:
!pip install openAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd

from datetime import datetime

from openai import OpenAI
from bs4 import BeautifulSoup

client = OpenAI(
    api_key="..."
)

In [ ]:
# player_name = 'Francisco Conceição'
# url = 'https://fbref.com/en/players/5ef3d210/Francisco-Conceicao'
# attrs_percentile_stats = 'scout_summary_AM'
# attrs_sim_players = 'similar_AM'

player_name = 'Francisco Trincão'
url = 'https://fbref.com/en/players/77e39b04/Francisco-Trincao'
attrs_percentile_stats = 'scout_summary_AM' #MF' #DF
attrs_sim_players = 'similar_AM'

df = pd.read_html(
    url,
    attrs={'id': attrs_percentile_stats}
)[0]

df1 = pd.read_html(
    url,
    attrs={'id': attrs_sim_players}
)[0]

In [ ]:
df1['Player_Club'] = df1['Player'] + ' (' + df1['Squad'] + ')'

sim_players = list(df1['Player_Club'][:6])
sim_players

['Steven Bergwijn (Ajax)',
 'Calvin Stengs (Feyenoord)',
 'Rafa Silva (Benfica)',
 'Juan Brunetta (Tigres UANL)',
 'Ricardo Horta (Braga)',
 'Pedro Gonçalves (Sporting CP)']

In [ ]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
position = soup.select_one('p:-soup-contains("Position")').text.split(':')[-2].split(' ')[0].strip()
birthday = soup.select_one('span[id="necro-birth"]').text.strip()

height = soup.select('#meta > div > p:nth-child(3) > span:nth-child(1)')[0].text.split('<span>')[0]
weight = soup.select('#meta > div > p:nth-child(3) > span:nth-child(2)')[0].text
foot = soup.select_one('p:-soup-contains("Footed")').text.split('Footed')[1].split(': ')[1]

age = (datetime.now() - datetime.strptime(birthday, '%B %d, %Y')).days // 365
team = soup.select_one('p:-soup-contains("Club")').text.split(':')[-1].strip()

In [ ]:
media_item_div = soup.find('div', {'class': 'media-item'})
img_tag = media_item_div.find('img') if media_item_div else None

player_image_url = img_tag['src'] if img_tag else 'URL default da imagem se não encontrada'

In [ ]:
# df.columns = df.columns.droplevel(0)

df = df.dropna(subset='Statistic')

In [ ]:
df.head()

,Statistic,Per 90,Percentile
0,Non-Penalty Goals,0.49,97.0
1,npxG: Non-Penalty xG,0.34,91.0
2,Shots Total,3.16,90.0
3,Assists,0.29,85.0
4,xAG: Exp. Assisted Goals,0.26,83.0


In [ ]:
prompt = f"""
I need you to create a scouting report on {player_name}. Can you provide me with a summary of their strengths and weaknesses?

Here is the data I have on him:

Here is the data I have on him:

Player: {player_name}
Height: {height}
Weight: {weight}
Position: {position}
Age: {age}
Team: {team}

List of similar players to {player_name} and respective clubs.
{df1.to_markdown()}

{df.to_markdown()}

Return the scouting report in the following HTML:


<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Report for {player_name}</title>
</head>
<body>
  <div style="position: relative; padding: 30px; display: flex; align-items: center;">
    <div style="flex: 0 0 auto;">
      <img src="{player_image_url}" alt=" {player_name} headshot">
    </div>
    <div style="margin-left: 20px;">
      <h1 style="color: darkblue; font-size: 28px;">Report for {player_name}</h1>
      <p>
        <span style="font-weight: bold;">Player:</span> {player_name}<br>
        <span style="font-weight: bold;">Height:</span> {height}<br>
        <span style="font-weight: bold;">Weight:</span> {weight}<br>
        <span style="font-weight: bold;">Position:</span> {position}<br>
        <span style="font-weight: bold;">Age:</span> {age}<br>
        <span style="font-weight: bold;">Team:</span> {team}
      </p>
    </div>
    <div style="flex: 1;"></div>
  </div>
  <div style="padding: 0 30px;">
    <h1 style="color: darkblue; text-align: center;">Summary</h1>
    <p>
      <a brief summary of the player's overall performance and if he would be beneficial to the team>
    </p>
    <h2 style="color: darkblue; text-align: left;">Strengths</h2>
    <ul>
      <li><i>a list of 1 to 3 strengths</i></li>
    </ul>
    <h2 style="color: darkblue; text-align: left;">Weaknesses</h2>
    <ul>
      <li><i>a list of 1 to 3 weaknesses</i></li>
    </ul>
    <h2 style="color: darkblue; text-align: left;">Potential</h2>
    <p>
      < assessment of the player's potential for growth >
    </p>
    <h2 style="color: darkblue; text-align: center;">Similar players</h2>
      <p> < mention the similar players to {player_name} ></p>
    <div style="text-align: center; margin-top: 50px;">
      <img src="https://raw.githubusercontent.com/ricardoandreom/Data/main/Images/Personal%20Logos/Half%20Space%20Preto.png" alt="Logo" style="width: 200px;">
    </div>
  </div>
</body>
</html>

"""

In [ ]:
print(prompt)


I need you to create a scouting report on Francisco Trincão. Can you provide me with a summary of their strengths and weaknesses?

Here is the data I have on him:

Here is the data I have on him:

Player: Francisco Trincão
Height: 186cm
Weight: 77kg
Position: FW-MF
Age: 24
Team: Sporting CP

List of similar players to Francisco Trincão and respective clubs.
|    |   Rk | Player             | Nation   | Squad       | Compare   | Player_Club                      |
|---:|-----:|:-------------------|:---------|:------------|:----------|:---------------------------------|
|  0 |    1 | Steven Bergwijn    | nl NED   | Ajax        | Compare   | Steven Bergwijn (Ajax)           |
|  1 |    2 | Calvin Stengs      | nl NED   | Feyenoord   | Compare   | Calvin Stengs (Feyenoord)        |
|  2 |    3 | Rafa Silva         | pt POR   | Benfica     | Compare   | Rafa Silva (Benfica)             |
|  3 |    4 | Juan Brunetta      | ar ARG   | Tigres UANL | Compare   | Juan Brunetta (Tigres UANL)     

In [ ]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "You are a professional football (soccer) scout."},
        {"role": "user", "content": prompt},
    ],
)

In [ ]:
#print(response.choices[0].message.content)

In [ ]:
!pip install weasyprint

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.1/289.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.9/848.9 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.6 MB/s eta 0:00:00


In [ ]:
from weasyprint import HTML

html = response.choices[0].message.content#.split('```html')[1].split('```')[0]

output_pdf = player_name + "_scouting_report.pdf"

with open("temp.html", "w") as f:
    f.write(html)

HTML("temp.html").write_pdf(output_pdf)

import os
os.remove("temp.html")


In [ ]:
response.choices[0].message.content#.split('```html')[1].split('```')[0]

'<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\n<title>Report for João Neves</title>\n</head>\n<body>\n  <div style="position: relative; padding: 30px; display: flex; align-items: center;">\n    <div style="flex: 0 0 auto;">\n      <img src="https://fbref.com/req/202302030/images/headshots/5ef3d210_2022.jpg" alt="João Neves headshot">\n    </div>\n    <div style="margin-left: 20px;">\n      <h1 style="color: darkblue; font-size: 28px;">Report for João Neves</h1>\n      <p>\n        <span style="font-weight: bold;">Player:</span> João Neves<br>\n        <span style="font-weight: bold;">Height:</span> 186cm<br>\n        <span style="font-weight: bold;">Weight:</span> 77kg<br>\n        <span style="font-weight: bold;">Position:</span> FW-MF<br>\n        <span style="font-weight: bold;">Age:</span> 24<br>\n        <span style="font-weight: bold;">Team:</span> Sporting CP\n      </p>\n    </div>\n   

In [ ]:
#response.choices[0].message.content.split('```html')[1].split('```')[0]


'\n<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\n<title>Report for Francisco Conceição</title>\n</head>\n<body>\n  <div style="position: relative; padding: 30px;">\n    <h1 style="color: darkblue; text-align: center; position: relative; top: 0px; right: 0px; width: 500px;">Report for Francisco Conceição</h1>\n    <p style="position: absolute; top: 120px; right: 1000px; width: 200px;">\n      <span style="font-weight: bold;">Player:</span> Francisco Conceição<br>\n      <span style="font-weight: bold;">Height:</span> 170cm<br>\n      <span style="font-weight: bold;">Weight:</span> 63kg<br>\n      <span style="font-weight: bold;">Position:</span> FW-MF<br>\n      <span style="font-weight: bold;">Age:</span> 21<br>\n      <span style="font-weight: bold;">Team:</span> Porto\n    </p>\n    <div style="text-align: center;">\n      <img src="https://fbref.com/req/202302030/images/headshots/5ef3d210_20